import modules & 

In [24]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: /Users/haph/Study/ubion/Project1/mid_code


load data

In [25]:
jpy = pd.read_csv('../dataset/JPY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [26]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
jpy['jpy_yesterday_change'] = round((jpy['close']-jpy['open'])/jpy['open']*100, 3)
jpy['jpy_yesterday_change'] = jpy['jpy_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
jpy['close_shift'] = jpy['close'].shift(1)
jpy['jpy_night_change'] = round((jpy['open']-jpy['close_shift'])/jpy['close_shift']*100, 3)

jpy.drop('close_shift',axis=1,inplace=True)

In [27]:
jpy.head(10)

,date,close,open,jpy_yesterday_change,jpy_night_change
0,2023- 10- 27,9.0101,8.9875,NaN,NaN
1,2023- 10- 26,8.9934,9.0070,0.251,-0.034
2,2023- 10- 25,9.0007,8.9674,-0.151,-0.289
3,2023- 10- 24,8.9640,8.9736,0.371,-0.301
4,2023- 10- 23,8.9636,9.0147,-0.107,0.566
5,2023- 10- 20,9.0017,9.0605,-0.567,1.081
6,2023- 10- 19,9.0528,9.0396,-0.649,0.421
7,2023- 10- 18,9.0293,9.0347,0.146,-0.200
8,2023- 10- 17,9.0301,9.0180,-0.060,-0.125
9,2023- 10- 16,9.0146,9.0518,0.134,0.240


In [28]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  66 non-null     object 
 1   close                 66 non-null     float64
 2   open                  66 non-null     float64
 3   jpy_yesterday_change  65 non-null     float64
 4   jpy_night_change      65 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.7+ KB


기간 통일

In [29]:
ks200 = pd.read_csv('../dataset/clean_feature/ks200_present.csv', index_col=0)
ks200.date = pd.to_datetime(ks200.date)

In [30]:
# 데이터 타입 변경
jpy['date'] = pd.to_datetime(jpy['date'])

In [31]:
jpy = pd.merge(jpy, ks200, left_on='date', right_on='date', how='inner')[jpy.columns]

#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [32]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
jpy['jpy_yesterday_change'] = round((jpy['close']-jpy['open'])/jpy['open']*100, 3)
jpy['jpy_yesterday_change'] = jpy['jpy_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
jpy['close_shift'] = jpy['close'].shift(1)
jpy['jpy_night_change'] = round((jpy['open']-jpy['close_shift'])/jpy['close_shift']*100, 3)

jpy.drop('close_shift',axis=1,inplace=True)

In [33]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
# jpy = jpy.loc[jpy['date']<='2023-10-20',].loc[jpy['date']>='2023-07-31',]
jpy = jpy[['date','jpy_yesterday_change','jpy_night_change']]

In [34]:
# 날짜순으로 오름차순 정렬
jpy.sort_values(by='date',inplace=True)
jpy.reset_index(drop=True, inplace=True)

In [35]:
jpy

,date,jpy_yesterday_change,jpy_night_change
0,2023-07-31,0.102,0.420
1,2023-08-01,0.743,-0.874
2,2023-08-02,0.494,-1.270
3,2023-08-03,0.860,-1.427
4,2023-08-04,-0.304,-0.573
5,2023-08-07,0.024,0.203
6,2023-08-08,-0.542,0.477
7,2023-08-09,-0.462,0.972
8,2023-08-10,0.805,-0.390
9,2023-08-11,0.136,-0.961


In [36]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  54 non-null     datetime64[ns]
 1   jpy_yesterday_change  53 non-null     float64       
 2   jpy_night_change      53 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB


In [37]:
jpy['jpy_yesterday_change'].loc[jpy['date'] == '2023-10-20'] = -0.567
jpy['jpy_night_change'].loc[jpy['date'] == '2023-10-20'] = 1.081

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_41809/2647994037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jpy['jpy_yesterday_change'].loc[jpy['date'] == '2023-10-20'] = -0.567
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_41809/2647994037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jpy['jpy_night_change'].loc[jpy['date'] == '2023-10-20'] = 1.081


csv 파일로 새로 저장

In [38]:
jpy.to_csv("../dataset/clean_feature/jpy_krw_final.csv")